## Packages

In [1]:
import pickle
import pandas as pd

## Classes and Functions

## Dependencies

In [2]:
train = pd.read_csv('../../assets/data/splits/train/preprocessed.csv')
val = pd.read_csv('../../assets/data/splits/val/preprocessed.csv')

In [3]:
y_train = train['label']
y_val = val['label']

In [4]:
with open('../../assets/traditional_assets/cv_vec.pkl', 'rb') as fout:
    cv_vec = pickle.load(fout)


In [5]:
with open('../../assets/traditional_assets/tfidf_vec.pkl', 'rb') as fout:
    tfidf_vec = pickle.load(fout)

In [6]:
cv_vec.get_feature_names_out()

array(['abaixo', 'abertura', 'abicom', 'absurdo', 'aceita', 'acelera',
       'acelerar', 'acionistas', 'acompanha', 'acordo', 'acucar',
       'acumula', 'administrativo', 'aereas', 'afirma', 'aflicao',
       'afunda', 'agencia', 'agenda', 'agitam', 'agosto', 'ajudar',
       'ajusta', 'ajustar', 'ajuste', 'ajustes', 'alagoas', 'albacora',
       'alcanca', 'alerta', 'alertam', 'aliviar', 'alivio', 'altera',
       'alteracoes', 'amazonas', 'ambiente', 'ameaca', 'america',
       'americanas', 'amplia', 'analise', 'analistas', 'antecipa',
       'anulacao', 'anulou', 'anuncia', 'anuncio', 'apenas', 'apesar',
       'apetite', 'apoiado', 'aponta', 'aposta', 'apostando', 'apostar',
       'apostas', 'apreensao', 'aprova', 'aprovacao', 'aprovam',
       'aprovar', 'aproveitar', 'aproxima', 'aproximar', 'aquisicao',
       'aquisicoes', 'arbitragem', 'arezzo', 'argentina', 'arrendamento',
       'arrendar', 'assembleia', 'assina', 'assinam', 'assume', 'assumir',
       'assumira', 'ataqu

## Grid Params

### KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn_params = {
    # 'n_neighbors': [11, 21, 40, 60, 80, 100],
    'n_neighbors': [15 ,17 ,19, 21, 23, 25, 27],
    'metric': ['cosine'],
    'weights': ['distance']
    # 'weights': ['uniform', 'distance'],
    # 'metric': ['cosine']
}

### SVM

In [8]:
from sklearn.svm import SVC

svm_params = {
    # 'C': [1,10, 50, 100],
    'C': [1,5,10],
    # 'C': [1,5,10],
    # 'kernel': ['rbf']
    # 'kernel' : ['rbf'],
}

### Naive Bayes

In [9]:
from sklearn.naive_bayes import MultinomialNB

nb_params = {
    'alpha': [0.1, 1, 10],
    'fit_prior': [True, False]
}

### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

lr_params = {
    'penalty': ['l1','l2', None],
    'C': [0.1, 1, 10],
    'solver': ['liblinear','sag', 'saga']
}

## Tuning and Evaluation

In [11]:
import numpy as np
from sklearn.model_selection import PredefinedSplit

split_index = [-1] * len(train) + [0] * len(val)

X = pd.concat([train, val], axis=0, ignore_index=True)

y = np.concatenate((y_train, y_val), axis=0)
pds = PredefinedSplit(test_fold=split_index)





### Count Vectorizer

In [12]:
X_cv = cv_vec.transform(X['title']).toarray()
pd.DataFrame(X_cv, columns=cv_vec.get_feature_names_out())

,abaixo,abertura,abicom,absurdo,aceita,acelera,acelerar,acionistas,acompanha,acordo,...,vender,vermelho,vespera,vinculante,vinculantes,volatil,volatilidade,voltar,volume,warren
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1882,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import GridSearchCV

# from hypopt import GridSearch

model_params = ([KNeighborsClassifier(), SVC(), MultinomialNB(), LogisticRegression()],
                [knn_params, svm_params, nb_params, lr_params])

list_best_models_params = []
for model, params in zip(model_params[0], model_params[1]):
    gs = GridSearchCV(model,
                      param_grid=params,
                      )

    gs.fit(X_cv, y)
    print(f"Best CV results for {model.__class__.__name__}")
    print("Best Score of train set: " + str(gs.best_score_))
    print("Best estimator: " + str(gs.best_estimator_))
    print("Best parameter set: " + str(gs.best_params_))

    store_best_model_configs = {
        'model_name': model.__class__.__name__,
        'best_score': gs.best_score_,
        'best_estimator': gs.best_estimator_,
        'best_params': gs.best_params_
    }

    list_best_models_params.append(store_best_model_configs)

df_best_models_params = pd.DataFrame(list_best_models_params)
df_best_models_params.to_csv('../../assets/traditional_assets/best_models_params_cv.csv', index=False)

df_best_models_params
# cv_best_model = gs.best_estimator_
# print("Test Score: " + str(gs.score(X_val_cv, y_val)))
# print("----------------------------------------------------")

Best CV results for KNeighborsClassifier
Best Score of train set: 0.5657516174757553
Best estimator: KNeighborsClassifier(metric='cosine', n_neighbors=27, weights='distance')
Best parameter set: {'metric': 'cosine', 'n_neighbors': 27, 'weights': 'distance'}
Best CV results for SVC
Best Score of train set: 0.5805959047338358
Best estimator: SVC(C=1)
Best parameter set: {'C': 1}
Best CV results for MultinomialNB
Best Score of train set: 0.5774128808611567
Best estimator: MultinomialNB(alpha=10)
Best parameter set: {'alpha': 10, 'fit_prior': True}


C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Us

Best CV results for LogisticRegression
Best Score of train set: 0.5784738888187164
Best estimator: LogisticRegression(C=0.1, solver='liblinear')
Best parameter set: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Guilherme\AppData\Roaming\Python\Python310\si

,model_name,best_score,best_estimator,best_params
0,KNeighborsClassifier,0.565752,"KNeighborsClassifier(metric='cosine', n_neighb...","{'metric': 'cosine', 'n_neighbors': 27, 'weigh..."
1,SVC,0.580596,SVC(C=1),{'C': 1}
2,MultinomialNB,0.577413,MultinomialNB(alpha=10),"{'alpha': 10, 'fit_prior': True}"
3,LogisticRegression,0.578474,"LogisticRegression(C=0.1, solver='liblinear')","{'C': 0.1, 'penalty': 'l2', 'solver': 'libline..."


In [14]:

cv_best_model = gs.best_estimator_
cv_best_model

LogisticRegression(C=0.1, solver='liblinear')

### TF-IDF

In [15]:
# X_train_tfidf = tfidf_vec.transform(X_train['title'])
# X_val_tfidf = tfidf_vec.transform(X_val['title'])
# X_train_tfidf
X_tfidf = tfidf_vec.transform(X['title']).toarray()
pd.DataFrame(X_tfidf, columns=tfidf_vec.get_feature_names_out())

,abaixo,abertura,abicom,absurdo,aceita,acelera,acelerar,acionistas,acompanha,acordo,...,vender,vermelho,vespera,vinculante,vinculantes,volatil,volatilidade,voltar,volume,warren
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [16]:
from sklearn.model_selection import GridSearchCV

model_params = ([KNeighborsClassifier(), SVC(), MultinomialNB(), LogisticRegression()],
                [knn_params, svm_params, nb_params, lr_params])

list_best_models_params = []
for model, params in zip(model_params[0], model_params[1]):
    gs = GridSearchCV(model,
                      param_grid=params,
                      )
    gs.fit(X_tfidf, y)
    print(f"Best TF-IDF results for {model.__class__.__name__}")
    print("Best Score on train set: " + str(gs.best_score_))
    print("Best estimator: " + str(gs.best_estimator_))
    print("Best parameter set: " + str(gs.best_params_) + "\n")
    store_best_model_configs = {
        'model_name': model.__class__.__name__,
        'best_score': gs.best_score_,
        'best_estimator': gs.best_estimator_,
        'best_params': gs.best_params_
    }

    list_best_models_params.append(store_best_model_configs)

df_best_models_params = pd.DataFrame(list_best_models_params)
df_best_models_params.to_csv('../../assets/traditional_assets/best_models_params_tfidf.csv', index=False)
df_best_models_params


# decide_best_model =
# print("Test Score: " + str(gs.score(X_val, y_val)))
# print("----------------------------------------------------")


Best TF-IDF results for KNeighborsClassifier
Best Score on train set: 0.5684041373696547
Best estimator: KNeighborsClassifier(metric='cosine', n_neighbors=27, weights='distance')
Best parameter set: {'metric': 'cosine', 'n_neighbors': 27, 'weights': 'distance'}

Best TF-IDF results for SVC
Best Score on train set: 0.5779433848399366
Best estimator: SVC(C=1)
Best parameter set: {'C': 1}

Best TF-IDF results for MultinomialNB
Best Score on train set: 0.5779433848399366
Best estimator: MultinomialNB(alpha=10)
Best parameter set: {'alpha': 10, 'fit_prior': True}



C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Us

Best TF-IDF results for LogisticRegression
Best Score on train set: 0.5779433848399366
Best estimator: LogisticRegression(C=0.1, penalty='l1', solver='liblinear')
Best parameter set: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}



C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Guilherme\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Guilherme\AppData\Roaming\Python\Python310\si

,model_name,best_score,best_estimator,best_params
0,KNeighborsClassifier,0.568404,"KNeighborsClassifier(metric='cosine', n_neighb...","{'metric': 'cosine', 'n_neighbors': 27, 'weigh..."
1,SVC,0.577943,SVC(C=1),{'C': 1}
2,MultinomialNB,0.577943,MultinomialNB(alpha=10),"{'alpha': 10, 'fit_prior': True}"
3,LogisticRegression,0.577943,"LogisticRegression(C=0.1, penalty='l1', solver...","{'C': 0.1, 'penalty': 'l1', 'solver': 'libline..."


In [17]:
tfidf_best_model = gs.best_estimator_
tfidf_best_model

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')

## Outputs

In [18]:
with open('../../assets/traditional_assets/cv_set.pkl', 'wb') as fout:
    pickle.dump((cv_vec, cv_best_model), fout)

with open('../../assets/traditional_assets/tfidf_set.pkl', 'wb') as fout:
    pickle.dump((tfidf_vec, cv_best_model), fout)
